In [1]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.optimizers import Adam

2025-08-17 09:47:59.177020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-17 09:47:59.205087: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-17 09:47:59.205135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-17 09:47:59.221761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-17 09:48:00.197350: W tensorflow/compiler/tf

In [2]:
class CNN_Model:
    def __init__(self,input_shape = (64,64,3), batch_size = 32, epochs = 100):
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None
        self.history = None
        self.train_generator = None
        self.test_generator = None
        
    ## load the data and prprocess
    def load_data(self,train_dir, test_dir):
       
        self.train_generator = image_dataset_from_directory(
            train_dir,
            labels='inferred',
            image_size= (self.input_shape[0],self.input_shape[1]),
            batch_size= self.batch_size,
            label_mode = 'categorical'
        )

        self.test_generator = image_dataset_from_directory(
            test_dir,
            labels='inferred',
            image_size= (self.input_shape[0],self.input_shape[1]),
            batch_size= self.batch_size,
            label_mode = 'categorical'
        )

    ## Buildin the CNN model
    def Build_model(self):
        self.model = Sequential([
            
            ### preprocess the image 
            Rescaling(1./255),
            RandomRotation(0.2),
            RandomZoom(0.2),
            RandomFlip('Horizontal'),
            
            ## 1st convolutional layer 
            Conv2D(32,(3,3), activation= 'relu', input_shape = self.input_shape),
            MaxPooling2D(pool_size=2 , strides=2),

            ## 2nd convolutional layer
            Conv2D(64,(3,3), activation= 'relu', input_shape = self.input_shape),
            MaxPooling2D(pool_size=2 , strides=2),

            ## 3rd convolutional layer
            Conv2D(128,(3,3), activation= 'relu', input_shape = self.input_shape),
            MaxPooling2D(pool_size=2 , strides=2),

            Flatten(),

            ### Create a fully connected layer
            Dense(units=128, activation= 'relu'),
            Dropout(0.5),  ## a regularization of 50 % 
            Dense(units= 4, activation= 'softmax'),
        ])
        
        ## Compile model and ready for training 
        self.model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    

    ## Training the model 
    def Train_model(self):
        self.history = self.model.fit(
            self.train_generator,
            validation_data = self.test_generator,
            epochs = self.epochs)        
        
    def Evaluate_model(self):
        loss,accuracy = self.model.evaluate(self.test_generator)
        print(f'Test Loss : {loss:.2f}')
        print(f'Test Accuracy : {accuracy * 100:.2f}')

    
    def Save_model(self, path = 'Model.keras'):

        if self.model is None:
            raise valuerror('Model not trained')
        self.model.save(path)
        print('Model saved Successfully ')
        
    def load_saved_model(self, path):
        self.model = tf.keras.models.load_model(path)
        print('Model loaded successfully')
        self.model.summary()

In [18]:
if __name__ == '__main__' :
    train_dir =  'Brain Tumor Classification (MRI)/Training'
    test_dir =  'Brain Tumor Classification (MRI)/Testing'
    
    Model_trainer = CNN_Model(input_shape=(64,64,3),batch_size = 32, epochs= 100) 
    Model_trainer.load_data(train_dir=train_dir, test_dir=test_dir)
    Model_trainer.Build_model()
    Model_trainer.Train_model()
    Model_trainer.Evaluate_model()
    Model_trainer.Save_model()

Found 2875 files belonging to 4 classes.
Found 395 files belonging to 4 classes.
Epoch 1/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - accuracy: 0.3520 - loss: 1.3410 - val_accuracy: 0.2278 - val_loss: 1.6997
Epoch 2/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.5076 - loss: 1.1294 - val_accuracy: 0.2329 - val_loss: 1.8490
Epoch 3/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.5790 - loss: 1.0325 - val_accuracy: 0.2835 - val_loss: 1.9879
Epoch 4/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - accuracy: 0.6213 - loss: 0.9100 - val_accuracy: 0.2658 - val_loss: 2.5815
Epoch 5/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.5887 - loss: 0.9227 - val_accuracy: 0.3089 - val_loss: 2.6308
Epoch 6/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.6374 - loss: 0.8246 - val_accuracy: 0.3266 - val_loss: 2.3178
Epoch 7/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.6718 - loss: 0.7683 - val_accuracy: 0.3494 - val_loss: 2.5904
Epoch 8/100
90/90 ━━━

In [22]:
Model_trainer.load_saved_model('Model.keras')

Model loaded successfully


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)              │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_2 (RandomRotation)   │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom_2 (RandomZoom)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip_2 (RandomFlip)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,051,150 (7.82 MB)

 Trainable params: 683,716 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,367,434 (5.22 MB)